## Edit the bash array in the next cell and run it to choose which notebook-based tests to run:

In [ ]:
test_items=(
    ci-check.ipynb
)

## Run next cell to run each test off-screen:

In [ ]:
for atest in "${test_items[@]}"; do
    if [ -f "$atest" ] && \
        rm -f "$atest.out" && \
        cp "$atest" "$atest.out" ; then
            echo "Running $atest..."
            time jupyter nbconvert --execute  --inplace  --ExecutePreprocessor.timeout=-1 "$atest.out"
            echo "   ...finished $atest."
    else
        echo "Test $atest not found."
    fi
done

## Generate summary:

In [ ]:
for atest in "${test_items[@]}"; do
    if [ -f "$atest.out" ] ; then
       if jq '(.cells[].outputs)' "$atest.out" | grep -qF '[[FAILED]]' ; then
          printf "Test %-15s failed." "$atest"
       else
          printf "Test %-15s passed." "$atest"
       fi
    else
        printf "Test %-15s did not return results." "$atest"
    fi
done | tee ~/test-summary.txt

## Send summary email:

In [ ]:
source ~/jh_test.config
mail -s "Test Summary" "$ADMIN_MAIL" <~/test-summary.txt